## libs

In [11]:
#import matplotlib.pyplot as plt
#import pandas as pd
import numpy as np
import math as m
#from scipy.interpolate import interp1d
#import scipy.optimize
import scipy.integrate as integrate

## constants

ask: 
1) if radius value is correct; 
2) what are the limits for distances (r)?
3) if temperature or any other parameters will be changed;

In [58]:
freq_ion = 3.29 * 10**15 #Herz
freq_max = 5 * 10**16 #Herz
alpha = 2.59 * 10**(-22) #meter^3 per sec
temperature = 40000 #Kelvins
radius = 0.05 * 206264.8 * 149.6 * 10**9 #meters
step = 15 * 10**12 #meters

c = 3 * 10**8 #meters per sec , light speed
h = 6.626070040812 * 10**(-34) #J*sec, Planck constant
k = 1.380648528 * 10**(-23) #J/K, Boltzmann constant

In [59]:
r = np.arange(radius, 2 * radius, step = step)
freq = np.linspace (freq_ion, freq_max, 200)

## functions

ask:
1) which n(H), n(e), n_H or n(H+) we need to calculate tau?
2) what temperature used to calculate brightness?
3) can we always define n(e) = n(H+)?


In [60]:
def brightness(freq):
    """mean brightness"""
    return 2 * h * freq**3 / c**2 / (np.exp(h * freq / (k * temperature)) - 1) 
    
def sigma(freq):
    """return photoionization cross section"""
    return 6.3 * 10**(-18) * np.power (freq_ion / freq, 3)

def density_H(n_e):
    """number densities of ions as a function of r"""
    return n_e**2 * alpha / integate.trapz(4 * m.pi * intensity(freq) / (h * freq) * sigma(freq), freq)

def tau(freq, r):
    """opacity on distance r"""
    return integrate.trapz(density_H(r)*sigma(freq))

def intensity(freq,r):
    """mean intensity Iν as a function of frequency"""
    return 0.25 * brightness(freq) * (radius / r)**2 * np.exp(-tau(freq, r))


In [61]:
#integrate.trapz([4,4,4,4,4],[0,1,2,3,4])

In [62]:
PHI = integrate.trapz(m.pi * brightness(freq) * np.exp(-0.00001) * sigma(freq) / (h * freq), freq)
n_e = ..?

In [56]:
ksi = PHI / (n_e * alpha)
n_H = n_e / (1 + ksi)
n_H_plus = n_e * ksi / (1 + ksi)

ask:
1) in pres there is sqrt(n_e * n_ion). where comes this formula from and what is n_ion (if n_ion = n_e = n_H)? 

In [ ]:
n_e_new = np.sqrt(n_e * n_H)